<a href="https://colab.research.google.com/github/2024aa05228-mg/MLOpsAssignmentGroup77/blob/main/MLOps_Assignment2_Group77.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#### This block is added to download the dataset only once and to avoid manual download


from google.colab import userdata
import os
import json
from pathlib import Path
from kaggle.api.kaggle_api_extended import KaggleApi

DATASET = "bhavikjikadara/dog-and-cat-classification-dataset"
DATA_DIR = "data/raw"

# Read secrets from Colab
kaggle_username = userdata.get("KAGGLE_USERNAME")
kaggle_key = userdata.get("KAGGLE_KEY")

if not kaggle_username or not kaggle_key:
    raise RuntimeError("Kaggle secrets not found in Colab")

kaggle_creds = {
    "username": kaggle_username,
    "key": kaggle_key
}

# Create BOTH possible Kaggle config paths
paths = [
    Path.home() / ".kaggle",
    Path.home() / ".config" / "kaggle"
]

kaggle_config_dir = paths[1] # Set it to ~/.config/kaggle, which is the path mentioned in the error

for p in paths:
    p.mkdir(parents=True, exist_ok=True)
    kaggle_json = p / "kaggle.json"
    with open(kaggle_json, "w") as f:
        json.dump(kaggle_creds, f)
    os.chmod(kaggle_json, 0o600)

# Set KAGGLE_CONFIG_DIR environment variable to ensure Kaggle API finds the credentials
os.environ['KAGGLE_CONFIG_DIR'] = str(kaggle_config_dir)

print("kaggle.json written to all expected locations and KAGGLE_CONFIG_DIR set.")

# Authenticate
api = KaggleApi()
api.authenticate()

# Download dataset
os.makedirs(DATA_DIR, exist_ok=True)
print("Downloading dataset...")
api.dataset_download_files(DATASET, path=DATA_DIR, unzip=True)

print("Dataset downloaded to data/raw/")

kaggle.json written to all expected locations and KAGGLE_CONFIG_DIR set.
Dataset URL: https://www.kaggle.com/datasets/bhavikjikadara/dog-and-cat-classification-dataset


TypeError: KaggleHttpClient.call() got an unexpected keyword argument 'headers'

In [4]:
import os
from PIL import Image
from torchvision import transforms # Added this import statement

RAW_DIR = "/content/data/raw/PetImages"
PROCESSED_DIR = "data/processed"
IMG_SIZE = (224, 224)

os.makedirs(PROCESSED_DIR, exist_ok=True)

for label in ["cat", "dog"]: # Changed labels to lowercase
    input_dir = os.path.join(RAW_DIR, label.capitalize()) # Original raw data has 'Cat' and 'Dog'
    output_dir = os.path.join(PROCESSED_DIR, label)
    os.makedirs(input_dir, exist_ok=True) # Create raw input directories if they don't exist
    os.makedirs(output_dir, exist_ok=True)

    # Only proceed if input_dir is not empty to avoid errors on empty directories
    if os.listdir(input_dir):
        for img_name in os.listdir(input_dir):
            img_path = os.path.join(input_dir, img_name)
            try:
                img = Image.open(img_path).convert("RGB")
                img = img.resize(IMG_SIZE)
                img.save(os.path.join(output_dir, img_name))
            except Exception as e:
                print(f"Could not process image {img_name} in {input_dir}: {e}")
    else:
        print(f"Warning: '{input_dir}' is empty. No images to process for this label.")

print("Preprocessing completed")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Preprocessing completed


In [ ]:
#Imports & Model definition
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import mlflow
import mlflow.pytorch
import os

In [3]:
import torch # Keep existing imports for context
from torchvision import datasets, transforms # Add transforms import
from torch.utils.data import DataLoader, random_split

PROCESSED_DIR = "data/processed" # Define PROCESSED_DIR in this cell

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(PROCESSED_DIR, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import os

class BaselineCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 54 * 54, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.fc(self.conv(x))

model = BaselineCNN()
os.makedirs('models', exist_ok=True)
torch.save(model.state_dict(), "models/baseline_cnn.pt")

!pip install mlflow
!mlflow ui


import mlflow
import mlflow.pytorch

mlflow.set_experiment("cats_vs_dogs_baseline")

with mlflow.start_run():
    mlflow.log_param("learning_rate", 0.001)
    mlflow.log_param("epochs", 10)
    mlflow.log_param("batch_size", 32)

    mlflow.log_metric("train_loss", train_loss)
    mlflow.log_metric("val_accuracy", val_acc)

    mlflow.pytorch.log_model(model, "baseline_cnn")


    mlflow.log_artifact("outputs/confusion_matrix.png")
mlflow.log_artifact("outputs/loss_curve.png")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.4 MB/s eta 0:00:00
Backend store URI not provided. Using sqlite:///mlflow.db
Registry store URI not provided. Using backend store URI.
2026/02/04 17